In [1]:
import pandas as pd
import numpy as np
import ast
import re
import statistics
from sklearn.metrics import roc_auc_score

In [2]:
test_padchest = pd.read_csv('../Data/Data_splits/pathology_detection-test.csv', index_col=0)

In [37]:
files = "/home/data_shares/purrlab/physionet.org/files/chexmask-cxr-segmentation-data/0.2"
padchest_masks = pd.read_csv(files+ "/OriginalResolution/Padchest.csv")

/home/caap/.conda/envs/multi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [38]:
test_padchest = pd.merge(padchest_masks, test_padchest , how="inner", on= "ImageID")

In [3]:
preds_bbox_both_mask_inside_df = pd.read_csv("../Predictions/PD_preds_bbox_both_mask_inside.csv")

In [4]:
preds_bbox_both_mask_outside_df = pd.read_csv("../Predictions/PD_preds_bbox_both_mask_outside.csv")

In [5]:
preds_bbox_mask_inside_df = pd.read_csv("../Predictions/PD_preds_bbox_mask_inside.csv")

In [6]:
preds_bbox_mask_outside_df = pd.read_csv("../Predictions/PD_preds_bbox_mask_outside.csv")

In [7]:
preds_dilated_mask_1_inside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_1_inside.csv")

In [8]:
preds_dilated_mask_1_outside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_1_outside.csv")

In [9]:
preds_dilated_mask_2_inside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_2_inside.csv")

In [10]:
preds_dilated_mask_2_outside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_2_outside.csv")

In [11]:
preds_dilated_mask_3_inside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_3_inside.csv")

In [12]:
preds_dilated_mask_3_outside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_3_outside.csv")

In [13]:
preds_dilated_mask_4_inside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_4_inside.csv")

In [14]:
preds_dilated_mask_4_outside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_4_outside.csv")

In [15]:
preds_original_mask_inside_df = pd.read_csv("../Predictions/PD_preds_original_mask_inside.csv")

In [16]:
preds_original_mask_outside_df = pd.read_csv("../Predictions/PD_preds_original_mask_outside.csv")

In [17]:
preds_bbox_both_mask_inside_df

,Unnamed: 0,Model_name,Val_data,Preds_model1,Preds_model2,Preds_model3
0,0,"PD, Multiclass, Densenet, Imagenet, Fine-tuned",PadChest test set,[[1.30609833e-02 4.78702970e-03 4.23425436e-03...,[[4.55487594e-02 1.59274554e-03 1.64995845e-02...,[[1.04184067e-02 4.47855424e-03 5.36778290e-03...


In [18]:
# Function for reading the predictions, which are strings, as numpy arrays
def str2array(s):
    # Remove space after [
    s=re.sub('\[ +', '[', s.strip())
    # Replace commas and spaces
    s=re.sub('[,\s]+', ', ', s)
    return np.array(ast.literal_eval(s))


# Function to arrange preds nicely in a df
def get_preds_PD(orig_pred_df, true_labels_df, print_auc=True):
    
    pathologies = ['Effusion', 'Pneumothorax', 'Atelectasis', 'Cardiomegaly', 'Pneumonia']
    all_preds = []
    
    for row_number in range(len(orig_pred_df)):
        for p_idx, p in enumerate(pathologies):
            preds = [[str2array(i["Preds_model1"]) for idx, i in orig_pred_df.iterrows()][row_number][:,p_idx],
                     [str2array(i["Preds_model2"]) for idx, i in orig_pred_df.iterrows()][row_number][:,p_idx],
                     [str2array(i["Preds_model3"]) for idx, i in orig_pred_df.iterrows()][row_number][:,p_idx]]
            all_preds.append(preds)
            
    # Constructing a df with the preds and 'true' labels
    preds_df = pd.DataFrame(list(zip(list(true_labels_df['ImagePath']),
                                     list(true_labels_df['Effusion']),
                                     list(true_labels_df['Pneumothorax']),
                                     list(true_labels_df['Atelectasis']),
                                     list(true_labels_df['Cardiomegaly']),
                                     list(true_labels_df['Pneumonia']),
                                     list(all_preds[0][0]),
                                     list(all_preds[0][1]),
                                     list(all_preds[0][2]),
                                     list(all_preds[1][0]),
                                     list(all_preds[1][1]),
                                     list(all_preds[1][2]),
                                     list(all_preds[2][0]),
                                     list(all_preds[2][1]),
                                     list(all_preds[2][2]),
                                     list(all_preds[3][0]),
                                     list(all_preds[3][1]),
                                     list(all_preds[3][2]),
                                     list(all_preds[4][0]),
                                     list(all_preds[4][1]),
                                     list(all_preds[4][2]))),
                            columns = ['ImagePath', 'Effusion', 'Pneumothorax', 'Atelectasis', 'Cardiomegaly',
                                       'Pneumonia', 'preds_Effusion_model1', 'preds_Effusion_model2', 'preds_Effusion_model3',
                                       'preds_Pneumothorax_model1', 'preds_Pneumothorax_model2', 'preds_Pneumothorax_model3',
                                       'preds_Atelectasis_model1', 'preds_Atelectasis_model2', 'preds_Atelectasis_model3',
                                       'preds_Cardiomegaly_model1', 'preds_Cardiomegaly_model2', 'preds_Cardiomegaly_model3',
                                       'preds_Pneumonia_model1', 'preds_Pneumonia_model2', 'preds_Pneumonia_model3'])


    # Computing the auc for each pathology separately
    for p in pathologies:
        if print_auc:
            print(p)
        auc_list = []
        for i in range(3):
            #print(i+1)
            auc = roc_auc_score(preds_df[p], preds_df['preds_' + str(p) + '_model' + str(i+1)])
            auc_list.append(auc)
        #print(auc_list)
        
        if print_auc:
            print("Average auc:", round(sum(auc_list)/len(auc_list)*100, 1))
        
            print()

    return preds_df

In [19]:
def create_auc_table(dataframes, names):
    pathologies = ['Effusion', 'Pneumothorax', 'Atelectasis', 'Cardiomegaly', 'Pneumonia']
    columns = ["name",'Effusion', 'Pneumothorax', 'Atelectasis', 'Cardiomegaly', 'Pneumonia']
    out_dict =  {pathology:list() for pathology in columns}

    for idx in range(len(dataframes)):
        out_dict["name"].append(names[idx])

        for p in pathologies:
            auc = roc_auc_score(dataframes[idx][p], dataframes[idx]['preds_' + str(p) + '_model1'])
            out_dict[p].append(auc)

    return pd.DataFrame(out_dict)

In [39]:
pds_bbox_both_mask_inside_df = get_preds_PD(preds_bbox_both_mask_inside_df, test_padchest)

Effusion
Average auc: 79.0

Pneumothorax
Average auc: 73.5

Atelectasis
Average auc: 76.3

Cardiomegaly
Average auc: 81.8

Pneumonia
Average auc: 69.3



In [40]:
pds_bbox_both_mask_outside_df = get_preds_PD(preds_bbox_both_mask_outside_df, test_padchest) 

Effusion
Average auc: 74.4

Pneumothorax
Average auc: 76.7

Atelectasis
Average auc: 66.3

Cardiomegaly
Average auc: 67.3

Pneumonia
Average auc: 63.9



In [41]:
pds_bbox_mask_inside_df = get_preds_PD(preds_bbox_mask_inside_df, test_padchest)

Effusion
Average auc: 87.9

Pneumothorax
Average auc: 79.8

Atelectasis
Average auc: 82.8

Cardiomegaly
Average auc: 87.8

Pneumonia
Average auc: 75.6



In [42]:
pds_bbox_mask_outside_df = get_preds_PD(preds_bbox_mask_outside_df, test_padchest)

Effusion
Average auc: 67.8

Pneumothorax
Average auc: 81.3

Atelectasis
Average auc: 62.0

Cardiomegaly
Average auc: 61.8

Pneumonia
Average auc: 63.8



In [43]:
pds_dilated_mask_1_inside_df = get_preds_PD(preds_dilated_mask_1_inside_df, test_padchest)

Effusion
Average auc: 67.4

Pneumothorax
Average auc: 51.9

Atelectasis
Average auc: 68.5

Cardiomegaly
Average auc: 74.1

Pneumonia
Average auc: 56.9



In [44]:
pds_dilated_mask_1_outside_df = get_preds_PD(preds_dilated_mask_1_outside_df, test_padchest)

Effusion
Average auc: 85.6

Pneumothorax
Average auc: 70.5

Atelectasis
Average auc: 79.3

Cardiomegaly
Average auc: 85.5

Pneumonia
Average auc: 68.1



In [45]:
pds_dilated_mask_2_inside_df = get_preds_PD(preds_dilated_mask_2_inside_df, test_padchest)

Effusion
Average auc: 68.8

Pneumothorax
Average auc: 54.3

Atelectasis
Average auc: 68.3

Cardiomegaly
Average auc: 72.0

Pneumonia
Average auc: 59.5



In [46]:
pds_dilated_mask_2_outside_df = get_preds_PD(preds_dilated_mask_2_outside_df, test_padchest)

Effusion
Average auc: 84.3

Pneumothorax
Average auc: 70.5

Atelectasis
Average auc: 77.7

Cardiomegaly
Average auc: 84.8

Pneumonia
Average auc: 67.6



In [47]:
pds_dilated_mask_3_inside_df = get_preds_PD(preds_dilated_mask_3_inside_df, test_padchest)

Effusion
Average auc: 71.2

Pneumothorax
Average auc: 55.6

Atelectasis
Average auc: 67.4

Cardiomegaly
Average auc: 70.7

Pneumonia
Average auc: 60.5



In [48]:
pds_dilated_mask_3_outside_df = get_preds_PD(preds_dilated_mask_3_outside_df, test_padchest)

Effusion
Average auc: 83.0

Pneumothorax
Average auc: 68.2

Atelectasis
Average auc: 75.6

Cardiomegaly
Average auc: 83.2

Pneumonia
Average auc: 67.1



In [49]:
pds_dilated_mask_4_inside_df = get_preds_PD(preds_dilated_mask_4_inside_df, test_padchest)

Effusion
Average auc: 72.7

Pneumothorax
Average auc: 53.3

Atelectasis
Average auc: 68.6

Cardiomegaly
Average auc: 70.1

Pneumonia
Average auc: 61.0



In [50]:
pds_dilated_mask_4_outside_df = get_preds_PD(preds_dilated_mask_4_outside_df, test_padchest)

Effusion
Average auc: 81.0

Pneumothorax
Average auc: 63.3

Atelectasis
Average auc: 73.8

Cardiomegaly
Average auc: 80.7

Pneumonia
Average auc: 65.8



In [51]:
pds_original_mask_inside_df = get_preds_PD(preds_original_mask_inside_df, test_padchest)

ValueError: malformed node or string: <_ast.Name object at 0x7f17b2cbc1d0>

In [ ]:
preds_bbox_both_mask_inside_df

,Unnamed: 0,Model_name,Val_data,Preds_model1,Preds_model2,Preds_model3
0,0,"PD, Multiclass, Densenet, Imagenet, Fine-tuned",PadChest test set,[[1.30609833e-02 4.78702970e-03 4.23425436e-03...,[[4.55487594e-02 1.59274554e-03 1.64995845e-02...,[[1.04184067e-02 4.47855424e-03 5.36778290e-03...


In [34]:
test_padchest

,ImageID,ImagePath,Labels,Effusion,Pneumothorax,Atelectasis,Cardiomegaly,Pneumonia,Chest_drain_tube,NSG_tube,Endotracheal_tube,Tracheostomy_tube
0,122000277774965864047324735331704213150_58hj8t...,/home/data_shares/purrlab_students/padchest-pr...,"['hemidiaphragm elevation', 'azygos lobe']",0,0,0,0,0,0,0,0,0
1,323329299430220123499459743740529233555_8lonth...,/home/data_shares/purrlab_students/padchest-pr...,"['costophrenic angle blunting', 'interstitial ...",0,0,0,0,0,0,0,0,0
2,216840111366964013451228379692012255095015356_...,/home/data_shares/purrlab_students/padchest-pr...,['normal'],0,0,0,0,0,0,0,0,0
3,215901993415805092612284930988632096429_2rbjm0...,/home/data_shares/purrlab_students/padchest-pr...,['callus rib fracture'],0,0,0,0,0,0,0,0,0
4,216840111366964013686042548532013283151248574_...,/home/data_shares/purrlab_students/padchest-pr...,"['pacemaker', 'dual chamber device']",0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9784,216840111366964012373310883942009152114440525_...,/home/data_shares/purrlab_students/padchest-pr...,['normal'],0,0,0,0,0,0,0,0,0
9785,216840111366964012922382741642011011084257538_...,/home/data_shares/purrlab_students/padchest-pr...,"['pleural effusion', 'atelectasis']",1,0,1,0,0,0,0,0,0
9786,216840111366964013340662495472012124102633950_...,/home/data_shares/purrlab_students/padchest-pr...,['normal'],0,0,0,0,0,0,0,0,0
9787,263194312664368739577287449640129198627_anu1co...,/home/data_shares/purrlab_students/padchest-pr...,['normal'],0,0,0,0,0,0,0,0,0


In [ ]:
pds_original_mask_outside_df = get_preds_PD(preds_original_mask_outside_df, test_padchest)

ValueError: malformed node or string: <_ast.Name object at 0x7f8413cea590>